In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib
df= pd.read_csv("traindata.csv")
df["Date"] = pd.to_datetime(df["Date"]) 

# conventional avocado X regions X Year
# filter out all conventional avocado (type = conventional)
conventional_avo = df[df["type"].isin(['conventional'])]
# sort by average price
conventional_avo = conventional_avo.sort_values(by='AveragePrice')
# plot
organic_avo = df[df["type"].isin(['organic'])]
# sort by average price
organic_avo = organic_avo.sort_values(by='AveragePrice')

dummy_type = pd.get_dummies(df['type'])
# print sample
dummy_type.sample(2)
# concat
df = pd.concat([df, dummy_type], axis=1)
# print(df.sample(2))
df['region'] = df['region'].astype('category')
df.dtypes
df['region'] = df['region'].cat.codes
df['region'].sample(3)
df['Date_Q'] = df['Date'].apply(lambda x: x.quarter)
# set the size of the figure
# plt.figure(figsize=(22,12))
# set the title
# plt.title("Correlation Matrix")

coe_col = ['AveragePrice', 'Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 
           'year', 'organic', 'conventional', 'Date_Q', 'region']
cm = np.corrcoef(df[coe_col].values.T)
sns.set(font_scale = 1.7)
# ax = sns.heatmap(cm,cbar = True, annot = True,square = True, fmt = '.2f', annot_kws = {'size':15}, yticklabels = coe_col, 
#                  xticklabels = coe_col)

Using matplotlib backend: Qt5Agg


In [2]:
from sklearn.model_selection import train_test_split
# split the dataframe to X and Y
X_columns = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'conventional', 'organic', 
             'Date_Q', 'year', 'region']
X = df[X_columns]
Y = df['AveragePrice']

print('X Shape:', X.shape)
print('Y Shape:', Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2019)

selected_features = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'conventional', 'organic']
X_train_sel = X_train[selected_features]
X_test_sel=X_test[selected_features]
# 
# 
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score
# 
model_2 = sm.OLS(y_train, X_train_sel)
res_2 = model_2.fit()
print(res_2.summary())

predictions = res_2.predict(X_test_sel)
print(explained_variance_score(predictions,y_test))
# Calculate R-squared
residuals = y_test - predictions
RMSE = np.sqrt(np.mean(residuals**2))
y_test_mean = np.mean(y_test)
tss =  np.sum((y_test - y_test_mean)**2 ) # total sum of square
rss =  np.sum(residuals**2) # sum of residuals
rsq =  1 - (rss/tss)
print('Model2 Selected features', rsq)

# built regression function



X Shape: (16325, 13)
Y Shape: (16325,)
                            OLS Regression Results                            
Dep. Variable:           AveragePrice   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.388
Method:                 Least Squares   F-statistic:                     1035.
Date:                Mon, 24 Jun 2019   Prob (F-statistic):               0.00
Time:                        22:37:13   Log-Likelihood:                -3183.9
No. Observations:               11427   AIC:                             6384.
Df Residuals:                   11419   BIC:                             6443.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Total Vol